### Assignment 3

## Question 1

In [65]:
import pandas as pd
import numpy as np
import pickle
import scipy.stats as stats

In [66]:
greene = pd.read_csv(r"../../_data/christensen_greene_f4.csv")

In [67]:
import statsmodels.formula.api as smf

In [3]:
greene

,id,YEAR,COST,Q,PL,SL,PK,SK,PF,SF
0,1,1970,0.2130,8.0,6869.47,0.3291,64.945,0.4197,18.000,0.2512
1,4,1970,3.0427,869.0,8372.96,0.1030,68.227,0.2913,21.067,0.6057
2,5,1970,9.4059,1412.0,7960.90,0.0891,40.692,0.1567,41.530,0.7542
3,14,1970,0.7606,65.0,8971.89,0.2802,41.243,0.1282,28.539,0.5916
4,15,1970,2.2587,295.0,8218.40,0.1772,71.940,0.1623,39.200,0.6606
...,...,...,...,...,...,...,...,...,...,...
153,214,1970,6.8293,946.6,10642.16,0.0883,43.600,0.1914,51.463,0.7203
154,215,1970,3.7605,377.0,7432.24,0.2117,74.120,0.2274,33.436,0.5609
155,216,1970,3.9822,391.0,5826.04,0.1926,78.288,0.0924,44.633,0.7151
156,217,1970,30.1880,5317.0,9586.63,0.0845,78.008,0.2009,41.840,0.7147


$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$

In [4]:
ct = greene.COST.values
q = greene.Q.values
p1 = greene.PL.values
p2 = greene.PF.values
p3 = greene.PK.values

In [5]:
lnct = np.log(ct)
lnq = np.log(q)
lnp1 = np.log(p1)
lnp2 = np.log(p2)
lnp3 = np.log(p3)

In [6]:
lnq_2 = np.multiply(lnq,lnq)
lnqlnp1 = np.multiply(lnq,lnp1)
lnqlnp2 = np.multiply(lnq,lnp2)
lnqlnp3 = np.multiply(lnq,lnp3)

In [7]:
lnp1_2 = np.multiply(lnp1,lnp1) 
lnp2_2 = np.multiply(lnp2,lnp2) 
lnp3_2 = np.multiply(lnp3,lnp3) 

In [8]:
lnp1lnp2 = np.multiply (lnp1,lnp2)
lnp1lnp3 = np.multiply (lnp1,lnp3)
lnp2lnp3 = np.multiply (lnp2,lnp3)

.
$$
\widehat{\boldsymbol{\beta}}^{(OLS)} = \left( \mathbf{X}^\top \mathbf{X}\right)^{-1} \mathbf{X}^\top \mathbf{Y}
$$
​


In [42]:
X_input_propio = pd.DataFrame(data=[lnq, lnq_2, lnqlnp1, lnqlnp2, lnqlnp3, lnp1, lnp2, lnp3, lnp1_2, lnp2_2, lnp3_2, lnp1lnp2, lnp1lnp3, lnp1lnp3],
                      index=["lnq", "(lnq)^2", "(lnq)(lnp1)", "(lnq)(lnp2)", "(lnq)(lnp3)", "lnp1", "lnp2", "lnp3", "(lnp1)^2", "(lnp2)^2", "(lnp3)^2", "(lnp1)(lnp2)", "(lnp1)(lnp3)", "(lnp2)(lnp3)"]).T
X_input_propio

,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
0,2.079442,4.324077,18.371538,6.010359,8.678634,8.834842,2.890372,4.173541,78.054437,8.354249,17.418442,25.535978,36.872574,36.872574
1,6.767343,45.796933,61.127805,20.624885,28.577410,9.032763,3.047708,4.222840,81.590803,9.288523,17.832381,27.529222,38.143915,38.143915
2,7.252762,52.602563,65.146469,27.026810,26.878966,8.982297,3.726416,3.706032,80.681665,13.886177,13.734670,33.471777,33.288677,33.288677
3,4.174387,17.425509,37.994654,13.989505,15.526556,9.101852,3.351272,3.719481,82.843703,11.231021,13.834542,30.502777,33.854168,33.854168
4,5.686975,32.341689,51.263140,20.863674,24.316554,9.014131,3.668677,4.275832,81.254554,13.459189,18.282743,33.069932,38.542913,38.542913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,6.852877,46.961918,63.543838,27.006249,25.870001,9.272579,3.940863,3.775057,85.980717,15.530402,14.251056,36.541963,35.004515,35.004515
154,5.932245,35.191533,52.877557,20.820004,25.542382,8.913583,3.509633,4.305685,79.451954,12.317525,18.538927,31.283405,38.379082,38.379082
155,5.968708,35.625470,51.749248,22.671977,26.025919,8.670093,3.798473,4.360394,75.170509,14.428401,19.013039,32.933118,37.805024,37.805024
156,8.578665,73.593485,78.650266,32.031471,37.375623,9.168125,3.733853,4.356811,84.054510,13.941657,18.981805,34.232428,39.943790,39.943790


In [43]:
y_input_propio = np.log(greene.COST)
y_input_propio

0     -1.546463
1      1.112745
2      2.241337
3     -0.273648
4      0.814789
         ...   
153    1.921222
154    1.324552
155    1.381834
156    3.407444
157    4.217391
Name: COST, Length: 158, dtype: float64

In [ ]:
def reg_ols( x : pd.DataFrame, y : pd.Series, intercept:bool=True) -> pd.DataFrame:
    if not isinstance( x , pd.DataFrame ):
        raise TypeError( "X variable is not .DataFrame.")
        
    if not isinstance( y, pd.Series ):
        raise TypeError( "Y variable is not pd.Series.")
        
    if not isinstance( intercept, bool):
        raise TypeError( "intercept is not boolean")
    
    if intercept:
        x_copia = x.copy()
        x_copia.insert(0, 'Intercept', 1)
        betas=np.linalg.inv(x_copia.T @ x_copia) @ x_copia.T @ y
        betas[ 'Parametros' ] =x_copia.columns.tolist()
        betas.set_index('Parametros', inplace = True)
        betas.rename(columns = {'ct': 'Coefs'}, inplace=True)
        return betas   
    else:
        betas=np.linalg.inv(x.T @ x) @ x.T @ y
        betas[ 'Parametros' ] =x.columns.tolist()
        betas.set_index('Parametros', inplace = True)
        betas.rename(columns = {'ct': 'Coefs'}, inplace=True)
        return betas 

In [ ]:
reg_ols( X_input_propio, y_input_propio, intercept=True )

In [26]:
beta_OLS = results.fit().summary2().tables[1].iloc[ :, [0]]
beta_OLS

,Coef.
Intercept,-76.259261
lnq,-1.080425
lnq_2,0.026489
lnqlnp1,0.131041
lnqlnp2,0.058652
lnqlnp3,0.040144
lnp1,14.718293
lnp2,-0.894733
lnp3,6.380797
lnp1_2,-0.769264


## Question 2

In [ ]:
## Matriz de varianzas
# Función en valor de x, y e intercepto base de datos panda
# y (endog) serie pandas
# intercepto puede ser verdadero o falso
# default verdadero
# cuando falso no incluir intercepto
# especificar parametros y si no se cumple requisito que salga error
# Hallarlo como función

In [69]:
#agregar una columna np a un panda
intercept=True

In [72]:
def reg_ols( x : pd.DataFrame, y : pd.Series, intercept) -> pd.DataFrame:
    if not isinstance( x , pd.DataFrame ):
        raise TypeError( "X variable is not .DataFrame.")
        
    if not isinstance( y, pd.Series ):
        raise TypeError( "Y variable is not pd.Series.")
        
    base_datos = x
    base_datos['log(Cost)'] = y.tolist()
    
    if intercept == True:
        regresion = smf.ols( 'base_datos.iloc[:, 14] ~ base_datos.iloc[:, 0]+base_datos.iloc[:, 1]+base_datos.iloc[:, 2]+base_datos.iloc[:, 3]+base_datos.iloc[:, 4]+base_datos.iloc[:, 5]+base_datos.iloc[:, 6]+base_datos.iloc[:, 7]+base_datos.iloc[:, 8]+base_datos.iloc[:, 9]+base_datos.iloc[:, 10]+base_datos.iloc[:, 11]+base_datos.iloc[:, 12]+base_datos.iloc[:, 13]', data = base_datos)
        result = regresion.fit().summary2().tables[1].iloc[ :, [0]]
    
    else:
        regresion = smf.ols( 'base_datos.iloc[:, 14] ~ base_datos.iloc[:, 0]+base_datos.iloc[:, 1]+base_datos.iloc[:, 2]+base_datos.iloc[:, 3]+base_datos.iloc[:, 4]+base_datos.iloc[:, 5]+base_datos.iloc[:, 6]+base_datos.iloc[:, 7]+base_datos.iloc[:, 8]+base_datos.iloc[:, 9]+base_datos.iloc[:, 10]+base_datos.iloc[:, 11]+base_datos.iloc[:, 12]+base_datos.iloc[:, 13]-1', data = base_datos)
        result = regresion.fit().summary2().tables[1].iloc[ :, [0]]
        
    return result

In [73]:
reg_ols( X_input_propio, y_input_propio, intercept = False )

,Coef.
"base_datos.iloc[:, 0]",-1.071221
"base_datos.iloc[:, 1]",0.025870
"base_datos.iloc[:, 2]",0.129634
"base_datos.iloc[:, 3]",0.059170
"base_datos.iloc[:, 4]",0.042702
"base_datos.iloc[:, 5]",1.072938
"base_datos.iloc[:, 6]",-1.414531
"base_datos.iloc[:, 7]",-0.303022
"base_datos.iloc[:, 8]",-0.152841
"base_datos.iloc[:, 9]",-0.049624


In [74]:
reg_ols( X_input_propio, y_input_propio, intercept = True )

,Coef.
Intercept,-76.181090
"base_datos.iloc[:, 0]",-1.081839
"base_datos.iloc[:, 1]",0.026470
"base_datos.iloc[:, 2]",0.131007
"base_datos.iloc[:, 3]",0.058108
"base_datos.iloc[:, 4]",0.041020
"base_datos.iloc[:, 5]",14.770618
"base_datos.iloc[:, 6]",-1.040195
"base_datos.iloc[:, 7]",6.352056
"base_datos.iloc[:, 8]",-0.771899
